"""
==============================================================================
04 - TRAIN MODELS FOR ADDITIONAL HORIZONS (3 and 5 bars ahead)
==============================================================================

Train XGBoost, CNN-LSTM, and TCN-Attention for horizons 3 and 5.
Horizon 1 models trained in notebooks 02, 03, 03b.

Each training cell is independent after data loading.
"""

# 04 - Train Additional Horizons

Train all 3 model types for horizons 3 and 5.

In [1]:
# ==============================================================================
# IMPORTS
# ==============================================================================

import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

from src.data.loader import load_and_merge_data, split_data_by_time
from src.labeling.oracle import create_oracle_labels
from src.features.builder import prepare_features
from src.features.indicators import get_indicator_columns
from src.models.xgb import XGBBaseline
from src.models.cnn_lstm import CNNLSTMModel
from src.models.tcn_attention import TCNAttentionModel

print("✅ Imports loaded")

✅ Imports loaded


In [2]:
# ==============================================================================
# CONFIGURATION
# ==============================================================================

SIGMA = 4
THRESHOLD = 0.0002
TRAIN_END = "2025-06-30"
TEST_START = "2025-07-01"
MODEL_DIR = 'models_artifacts'

# Load best TCN params if available (from grid search)
try:
    with open(f'{MODEL_DIR}/tcn_attention_best_params.json', 'r') as f:
        TCN_BEST_PARAMS = json.load(f)
    print(f"📋 Loaded TCN best params from grid search")
except:
    # Default params if grid search not run
    TCN_BEST_PARAMS = {
        'tcn_filters': 64,
        'num_tcn_blocks': 3,
        'lookback': 32,
        'dropout': 0.2,
        'use_class_weights': False
    }
    print(f"📋 Using default TCN params (run 03b for tuned params)")

all_results = {}

print(f"   Oracle: sigma={SIGMA}, threshold={THRESHOLD}")

📋 Loaded TCN best params from grid search
   Oracle: sigma=4, threshold=0.0002


In [3]:
# ==============================================================================
# LOAD DATA
# ==============================================================================

print("=" * 60)
print("📥 LOADING DATA")
print("=" * 60)

df = load_and_merge_data(end_date='2025-12-31')
df = create_oracle_labels(df, sigma=SIGMA, threshold=THRESHOLD)
print(f"✅ Loaded {len(df):,} rows")

📥 LOADING DATA

📥 LOADING DATA

1️⃣ Loading klines...
   Loaded 70,752 rows

2️⃣ Loading funding rates...
   Merged funding rates

3️⃣ Loading volume breakdown...
   Merged volume breakdown
⏰ Filtered until: 2025-12-31

📊 Data Summary:
   Rows: 70,081
   Columns: 16
   Date range: 2024-01-01 00:00:00+00:00 to 2025-12-31 00:00:00+00:00
   Columns: ['time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'taker_buy_volume', 'taker_buy_quote_volume', 'count', 'funding_interval_hours', 'last_funding_rate', 'buy_vol', 'sell_vol', 'total_vol', 'log_return']
✅ Loaded 70,081 rows


---
## HORIZON 3 (45 minutes ahead)
---

In [4]:
# ==============================================================================
# PREPARE DATA FOR HORIZON 3
# ==============================================================================

HORIZON = 3
print(f"\n{'='*60}")
print(f"🎯 PREPARING DATA FOR HORIZON={HORIZON}")
print("="*60)

all_results[HORIZON] = {}

df_features_h3, _ = prepare_features(df.copy(), horizon=HORIZON)
train_df_h3, val_df_h3, test_df_h3 = split_data_by_time(
    df_features_h3, train_end=TRAIN_END, test_start=TEST_START, val_ratio=0.1
)

feature_cols = get_indicator_columns(
    df_features_h3, exclude_cols=['time', 'target', 'smoothed_close', 'smooth_slope']
)
feature_cols = [c for c in feature_cols if c in train_df_h3.columns]

X_train_h3 = np.nan_to_num(train_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_train_h3 = train_df_h3['target'].values.astype(int)
X_val_h3 = np.nan_to_num(val_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_val_h3 = val_df_h3['target'].values.astype(int)
X_test_h3 = np.nan_to_num(test_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_test_h3 = test_df_h3['target'].values.astype(int)

print(f"   Train: {len(train_df_h3):,}, Val: {len(val_df_h3):,}, Test: {len(test_df_h3):,}")


🎯 PREPARING DATA FOR HORIZON=3

🔧 PREPARING FEATURES
Groups to add: ['momentum', 'overlap', 'trend', 'volatility', 'volume', 'statistics', 'candle']

📊 Adding indicators...
  -> Processing group: momentum
     Added 14 features: CCI_14_0.015, TSI_13_25_13, STOCHk_14_3_3, STOCHRSIk_14_14_3_3, STOCHRSId_14_14_3_3...
  -> Processing group: overlap
[!] VWAP requires an ordered DatetimeIndex.
     Added 7 features: EMA_10, HMA_9, EMA_100, EMA_20, EMA_50...
  -> Processing group: trend
     Added 17 features: CKSPl_10_3_20, AROONOSC_14, ADXR_14_2, AROONU_14, TRIX_30_9...
  -> Processing group: volatility
       BBL_20_2.0_2.0  BBM_20_2.0_2.0  BBU_20_2.0_2.0  BBB_20_2.0_2.0  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN       

In [5]:
# ==============================================================================
# TRAIN XGBOOST FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🌲 TRAINING XGBOOST (H={HORIZON})")
print("─"*60)

xgb_h3 = XGBBaseline(n_classes=3, device='cuda', random_state=42)
xgb_h3.fit(X_train_h3, y_train_h3, X_val_h3, y_val_h3, feature_names=feature_cols)
xgb_h3.tune(X_train_h3, y_train_h3, n_iter=15, cv_splits=3, scoring='f1_weighted')
xgb_metrics_h3 = xgb_h3.evaluate(X_test_h3, y_test_h3)

print(f"✅ Acc={xgb_metrics_h3['accuracy']:.4f}, F1={xgb_metrics_h3['f1_weighted']:.4f}")
Path(MODEL_DIR).mkdir(exist_ok=True)
xgb_h3.save(MODEL_DIR, name='xgb_baseline_h3')
all_results[3]['XGBoost'] = xgb_metrics_h3


────────────────────────────────────────────────────────────
🌲 TRAINING XGBOOST (H=3)
────────────────────────────────────────────────────────────

🔧 Tuning hyperparameters with TimeSeriesSplit...
Fitting 3 folds for each of 15 candidates, totalling 45 fits

✅ Best CV score: 0.4952
Best parameters: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
✅ Acc=0.5357, F1=0.5188
✅ Model saved to models_artifacts


In [6]:
# ==============================================================================
# TRAIN CNN-LSTM FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🧠 TRAINING CNN-LSTM (H={HORIZON})")
print("─"*60)

cnn_h3 = CNNLSTMModel(
    n_classes=3, lookback=32, conv_filters=64, lstm_units=64,
    dropout=0.3, learning_rate=0.0007, device='cuda'
)
cnn_h3.fit(X_train_h3, y_train_h3, X_val_h3, y_val_h3, epochs=30, batch_size=128, patience=5)
cnn_metrics_h3 = cnn_h3.evaluate(X_test_h3, y_test_h3)

print(f"✅ Acc={cnn_metrics_h3['accuracy']:.4f}, F1={cnn_metrics_h3['f1_weighted']:.4f}")
cnn_h3.save(MODEL_DIR, name='cnn_lstm_h3')
all_results[3]['CNN-LSTM'] = cnn_metrics_h3


────────────────────────────────────────────────────────────
🧠 TRAINING CNN-LSTM (H=3)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=32)...
   Train sequences: (46915, 32, 62)
   Val sequences: (5184, 32, 62)

🏗️ Building model...


I0000 00:00:1769122308.294849   52647 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21720 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "cnn_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 32, 62)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 64)         │        11,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv1 (Activation)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv2 (BatchNormalization)   │ (None, 32, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv2 (Activation)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv2 (Dropout)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_lstm1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_lstm1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 190,147 (742.76 KB)

 Trainable params: 189,635 (740.76 KB)

 Non-trainable params: 512 (2.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.4608 - loss: 1.0407 - val_accuracy: 0.4506 - val_loss: 1.0663 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5022 - loss: 0.9946 - val_accuracy: 0.4574 - val_loss: 1.0874 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5375 - loss: 0.9532 - val_accuracy: 0.4437 - val_loss: 1.1207 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5717 - loss: 0.9045 - val_accuracy: 0.4182 - val_loss: 1.1607 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.6094 - loss: 0.8502 - val_accuracy: 0.4041 - val_loss: 1.2229 - learning_rate: 7.0000e-04
Epoch 6/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6309 - loss: 0.8096
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
36

In [7]:
# ==============================================================================
# TRAIN TCN-ATTENTION FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"⚡ TRAINING TCN-ATTENTION (H={HORIZON})")
print("─"*60)

tcn_h3 = TCNAttentionModel(
    n_classes=3,
    lookback=TCN_BEST_PARAMS.get('lookback', 32),
    tcn_filters=TCN_BEST_PARAMS.get('tcn_filters', 64),
    num_tcn_blocks=TCN_BEST_PARAMS.get('num_tcn_blocks', 3),
    dropout=TCN_BEST_PARAMS.get('dropout', 0.2),
    learning_rate=0.0007,
    device='cuda'
)
tcn_h3.fit(
    X_train_h3, y_train_h3, X_val_h3, y_val_h3,
    epochs=30, batch_size=128, patience=5,
    use_class_weights=TCN_BEST_PARAMS.get('use_class_weights', False)
)
tcn_metrics_h3 = tcn_h3.evaluate(X_test_h3, y_test_h3)

print(f"✅ Acc={tcn_metrics_h3['accuracy']:.4f}, F1={tcn_metrics_h3['f1_weighted']:.4f}")
tcn_h3.save(MODEL_DIR, name='tcn_attention_h3')
all_results[3]['TCN-Attention'] = tcn_metrics_h3


────────────────────────────────────────────────────────────
⚡ TRAINING TCN-ATTENTION (H=3)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=16)...
   Train sequences: (46931, 16, 62)
   Val sequences: (5200, 16, 62)

🏗️ Building TCN-Attention model...


Model: "tcn_attention"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 16, 62)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_1 (TCNBlock)          │ (None, 16, 128)        │        32,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_2 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_3 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention                       │ (None, 16, 128)        │        66,304 │
│ (MultiHeadSelfAttention)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling1D)    │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,627 (791.51 KB)

 Trainable params: 201,859 (788.51 KB)

 Non-trainable params: 768 (3.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
 25/367 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3567 - loss: 1.2244

I0000 00:00:1769122362.845187   52957 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


367/367 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.4288 - loss: 1.0961 - val_accuracy: 0.4308 - val_loss: 1.1033 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4663 - loss: 1.0629 - val_accuracy: 0.4533 - val_loss: 1.0956 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4872 - loss: 1.0391 - val_accuracy: 0.4467 - val_loss: 1.0983 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5038 - loss: 1.0177 - val_accuracy: 0.4260 - val_loss: 1.1186 - learning_rate: 7.0000e-04
Epoch 5/30
359/367 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5162 - loss: 1.0039
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5214 - loss: 0.9946 - val_accuracy: 0.4338 - val_loss: 1.1213 - learning_rate: 7.0000e-04
Epoch 6/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5592 - loss:

---
## HORIZON 5 (75 minutes ahead)
---

In [8]:
# ==============================================================================
# PREPARE DATA FOR HORIZON 5
# ==============================================================================

HORIZON = 5
print(f"\n{'='*60}")
print(f"🎯 PREPARING DATA FOR HORIZON={HORIZON}")
print("="*60)

all_results[HORIZON] = {}

df_features_h5, _ = prepare_features(df.copy(), horizon=HORIZON)
train_df_h5, val_df_h5, test_df_h5 = split_data_by_time(
    df_features_h5, train_end=TRAIN_END, test_start=TEST_START, val_ratio=0.1
)

feature_cols = get_indicator_columns(
    df_features_h5, exclude_cols=['time', 'target', 'smoothed_close', 'smooth_slope']
)
feature_cols = [c for c in feature_cols if c in train_df_h5.columns]

X_train_h5 = np.nan_to_num(train_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_train_h5 = train_df_h5['target'].values.astype(int)
X_val_h5 = np.nan_to_num(val_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_val_h5 = val_df_h5['target'].values.astype(int)
X_test_h5 = np.nan_to_num(test_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_test_h5 = test_df_h5['target'].values.astype(int)

print(f"   Train: {len(train_df_h5):,}, Val: {len(val_df_h5):,}, Test: {len(test_df_h5):,}")


🎯 PREPARING DATA FOR HORIZON=5

🔧 PREPARING FEATURES
Groups to add: ['momentum', 'overlap', 'trend', 'volatility', 'volume', 'statistics', 'candle']

📊 Adding indicators...
  -> Processing group: momentum
     Added 14 features: CCI_14_0.015, TSI_13_25_13, STOCHk_14_3_3, STOCHRSIk_14_14_3_3, STOCHRSId_14_14_3_3...
  -> Processing group: overlap
[!] VWAP requires an ordered DatetimeIndex.
     Added 7 features: EMA_10, HMA_9, EMA_100, EMA_20, EMA_50...
  -> Processing group: trend
     Added 17 features: CKSPl_10_3_20, AROONOSC_14, ADXR_14_2, AROONU_14, TRIX_30_9...
  -> Processing group: volatility
       BBL_20_2.0_2.0  BBM_20_2.0_2.0  BBU_20_2.0_2.0  BBB_20_2.0_2.0  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN       

In [9]:
# ==============================================================================
# TRAIN XGBOOST FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🌲 TRAINING XGBOOST (H={HORIZON})")
print("─"*60)

xgb_h5 = XGBBaseline(n_classes=3, device='cuda', random_state=42)
xgb_h5.fit(X_train_h5, y_train_h5, X_val_h5, y_val_h5, feature_names=feature_cols)
xgb_h5.tune(X_train_h5, y_train_h5, n_iter=15, cv_splits=3, scoring='f1_weighted')
xgb_metrics_h5 = xgb_h5.evaluate(X_test_h5, y_test_h5)

print(f"✅ Acc={xgb_metrics_h5['accuracy']:.4f}, F1={xgb_metrics_h5['f1_weighted']:.4f}")
xgb_h5.save(MODEL_DIR, name='xgb_baseline_h5')
all_results[5]['XGBoost'] = xgb_metrics_h5


────────────────────────────────────────────────────────────
🌲 TRAINING XGBOOST (H=5)
────────────────────────────────────────────────────────────

🔧 Tuning hyperparameters with TimeSeriesSplit...
Fitting 3 folds for each of 15 candidates, totalling 45 fits

✅ Best CV score: 0.4261
Best parameters: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
✅ Acc=0.4946, F1=0.4662
✅ Model saved to models_artifacts


In [10]:
# ==============================================================================
# TRAIN CNN-LSTM FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🧠 TRAINING CNN-LSTM (H={HORIZON})")
print("─"*60)

cnn_h5 = CNNLSTMModel(
    n_classes=3, lookback=32, conv_filters=64, lstm_units=64,
    dropout=0.3, learning_rate=0.0007, device='cuda'
)
cnn_h5.fit(X_train_h5, y_train_h5, X_val_h5, y_val_h5, epochs=30, batch_size=128, patience=5)
cnn_metrics_h5 = cnn_h5.evaluate(X_test_h5, y_test_h5)

print(f"✅ Acc={cnn_metrics_h5['accuracy']:.4f}, F1={cnn_metrics_h5['f1_weighted']:.4f}")
cnn_h5.save(MODEL_DIR, name='cnn_lstm_h5')
all_results[5]['CNN-LSTM'] = cnn_metrics_h5


────────────────────────────────────────────────────────────
🧠 TRAINING CNN-LSTM (H=5)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=32)...
   Train sequences: (46915, 32, 62)
   Val sequences: (5184, 32, 62)

🏗️ Building model...


Model: "cnn_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 32, 62)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 64)         │        11,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv1 (Activation)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv2 (BatchNormalization)   │ (None, 32, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv2 (Activation)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv2 (Dropout)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_lstm1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_lstm1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 190,147 (742.76 KB)

 Trainable params: 189,635 (740.76 KB)

 Non-trainable params: 512 (2.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.4436 - loss: 1.0519 - val_accuracy: 0.4180 - val_loss: 1.1080 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.4867 - loss: 1.0115 - val_accuracy: 0.3752 - val_loss: 1.1661 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5252 - loss: 0.9649 - val_accuracy: 0.3553 - val_loss: 1.2625 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.5679 - loss: 0.9089 - val_accuracy: 0.3669 - val_loss: 1.2837 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.6080 - loss: 0.8451 - val_accuracy: 0.3590 - val_loss: 1.4068 - learning_rate: 7.0000e-04
Epoch 6/30
364/367 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6310 - loss: 0.8054
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
367

In [11]:
# ==============================================================================
# TRAIN TCN-ATTENTION FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"⚡ TRAINING TCN-ATTENTION (H={HORIZON})")
print("─"*60)

tcn_h5 = TCNAttentionModel(
    n_classes=3,
    lookback=TCN_BEST_PARAMS.get('lookback', 32),
    tcn_filters=TCN_BEST_PARAMS.get('tcn_filters', 64),
    num_tcn_blocks=TCN_BEST_PARAMS.get('num_tcn_blocks', 3),
    dropout=TCN_BEST_PARAMS.get('dropout', 0.2),
    learning_rate=0.0007,
    device='cuda'
)
tcn_h5.fit(
    X_train_h5, y_train_h5, X_val_h5, y_val_h5,
    epochs=30, batch_size=128, patience=5,
    use_class_weights=TCN_BEST_PARAMS.get('use_class_weights', False)
)
tcn_metrics_h5 = tcn_h5.evaluate(X_test_h5, y_test_h5)

print(f"✅ Acc={tcn_metrics_h5['accuracy']:.4f}, F1={tcn_metrics_h5['f1_weighted']:.4f}")
tcn_h5.save(MODEL_DIR, name='tcn_attention_h5')
all_results[5]['TCN-Attention'] = tcn_metrics_h5


────────────────────────────────────────────────────────────
⚡ TRAINING TCN-ATTENTION (H=5)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=16)...
   Train sequences: (46931, 16, 62)
   Val sequences: (5200, 16, 62)

🏗️ Building TCN-Attention model...


Model: "tcn_attention"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 16, 62)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_1 (TCNBlock)          │ (None, 16, 128)        │        32,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_2 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_3 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention                       │ (None, 16, 128)        │        66,304 │
│ (MultiHeadSelfAttention)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling1D)    │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,627 (791.51 KB)

 Trainable params: 201,859 (788.51 KB)

 Non-trainable params: 768 (3.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.4249 - loss: 1.0997 - val_accuracy: 0.4352 - val_loss: 1.1090 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4512 - loss: 1.0730 - val_accuracy: 0.4331 - val_loss: 1.1070 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4703 - loss: 1.0517 - val_accuracy: 0.4317 - val_loss: 1.1279 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4917 - loss: 1.0296 - val_accuracy: 0.4060 - val_loss: 1.1718 - learning_rate: 7.0000e-04
Epoch 5/30
364/367 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5039 - loss: 1.0149
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5099 - loss: 1.0058 - val_accuracy: 0.4188 - val_loss: 1.1801 - learning_rate: 7.0000e-04
Epoch 6/30
367/367

---
## FINAL SUMMARY
---

In [12]:
# ==============================================================================
# SUMMARY
# ==============================================================================

print("\n" + "=" * 60)
print("📋 TRAINING COMPLETE - SUMMARY")
print("=" * 60)

print(f"\n{'Horizon':<10} {'Model':<15} {'Accuracy':<12} {'F1 Weighted':<12}")
print("─" * 50)

for h in [3, 5]:
    if h in all_results:
        for model in ['XGBoost', 'CNN-LSTM', 'TCN-Attention']:
            if model in all_results[h]:
                m = all_results[h][model]
                print(f"{h:<10} {model:<15} {m['accuracy']:.4f}       {m['f1_weighted']:.4f}")

print(f"""
{'─' * 50}

📁 Models saved to: {MODEL_DIR}/
   • xgb_baseline_h3/h5
   • cnn_lstm_h3/h5
   • tcn_attention_h3/h5

🔜 NEXT: Run 05_comparison for full comparison.
""")


📋 TRAINING COMPLETE - SUMMARY

Horizon    Model           Accuracy     F1 Weighted 
──────────────────────────────────────────────────
3          XGBoost         0.5357       0.5188
3          CNN-LSTM        0.4868       0.4402
3          TCN-Attention   0.4791       0.4577
5          XGBoost         0.4946       0.4662
5          CNN-LSTM        0.4551       0.4402
5          TCN-Attention   0.4715       0.4184

──────────────────────────────────────────────────

📁 Models saved to: models_artifacts/
   • xgb_baseline_h3/h5
   • cnn_lstm_h3/h5
   • tcn_attention_h3/h5

🔜 NEXT: Run 05_comparison for full comparison.

